## Forked from https://www.kaggle.com/code/abdurrafae/improved-code-interpretation

**Lewis:** the only changes in this notebook are those needed to run the original one with the new Kaggle evaluation API

Forked From  https://kaggle.com/code/xiaoz259/pure-rng/notebook

credits:
https://www.kaggle.com/code/olyatsimboy/aimo-openmath-mistral-baseline \
https://www.kaggle.com/code/aatiffraz/prompt-prediction-w-mixtral-mistral7b-gemma-llama \
https://www.kaggle.com/code/thedrcat/aimo-mixtral-baseline

In [1]:
import time
NOTEBOOK_START_TIME = time.time()

PRIVATE = False
TRAIN_PATH = 'train.csv'

In [2]:
TOTAL_TOKENS = 2048
n_repetitions = 10 # if PRIVATE else 4 # Original notebook had 22 but times out :(

TIME_LIMIT = 31500 if PRIVATE else 6400
PER_Q_TIME_LIMIT = 640

In [3]:
import pandas as pd
if not PRIVATE:
    class train_env():
        def __init__(self, randomize=False):
            self.randomlize = randomize
            
            self.df = pd.read_csv(TRAIN_PATH)
            self.df['ground_truth'] = self.df['answer']
            self.df['answer'] = -1
            
            if self.randomlize:
                self.df = self.df.reset_index().sample(frac=1).reset_index(drop=True)
            
            self.predict_called = True
            self.counter = 0
            self.len = len(self.df)
        
        
        def iter_test(self):
             while self.counter<self.len:
                if self.predict_called:
                    self.predict_called = False
                    yield (self.df.loc[[self.counter]][['id','problem']]),(self.df.loc[[self.counter]][['id','answer']])
                else:
                    print("You must call `predict()` successfully before you can continue with `iter_test()`")
                    yield None 
                
        def predict(self, answer):
            self.df.loc[self.counter, ('answer')] = answer['answer'].values[0]
            self.predict_called = True
            self.counter+=1

    env = train_env(randomize=True)
    iter_test = env.iter_test()
else:
    # Set up the evaluation API
    import aimo

    env = aimo.make_env()
    iter_test = env.iter_test()

TO-DO

Change temperature as the question goes longer
Change temperature based on question lenght

# Zero-shot MMOS-DeepSeekMath-7B with self-consistency and generated code reasoning evaluation

Self-consistency is a modification of the standard greedy decoding in reasoning pipelines via sampling several diverse answers followed by aggregation, e.g., most common answer ([SC-CoT paper](https://arxiv.org/pdf/2203.11171.pdf)).

In this kernel, we will consider MMOS-DeepSeekMath-7B RL-tuned backbone; in my experiments, this model produces more consistent code reasoning and the code block execution will allow us to decrease arithmetic hallucinations.

In [4]:
DEBUG = False
# QUANT = False

In [5]:
# !pip uninstall torch
# !pip install --no-index --find-links=/kaggle/input/vllm-whl -U vllm
# !pip install --no-index -U /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install --no-index -U /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl --find-links /kaggle/input/vllm-whl

In [6]:
MODEL_NAME = '/scratch/aakash_ks.iitr/models/deepseek/'
#MODEL_NAME = 'meta-llama/Meta-Llama-3-8B-Instruct'

In [7]:
stop_words = ["```output", "```python", "```\nOutput" , ")\n```" , "``````output"]
#stop_words = ["```output", "```python", "```\nOutput" , ")\n```" , "``````output", '\n\n', '<|eot_id|>',]

In [8]:
from vllm import LLM, SamplingParams

llm = LLM(
    model=MODEL_NAME,
    dtype="half",
    #enforce_eager=True, #to disable CUDA graphs for debugging 
    gpu_memory_utilization=0.94,  #with enforce eager you can use 0.99; else 0.91
    swap_space=2,  #CPU RAM per gpu preferable is 2, default is 4
    max_model_len=TOTAL_TOKENS,
    kv_cache_dtype="fp8",  #fp8_e5m2
    tensor_parallel_size=2, #2 in kaggle
    max_num_seqs=1, #default is 256 keep 20 while batching
    max_seq_len_to_capture=TOTAL_TOKENS//2,
    #disable_custom_all_reduce=True,
)

# if getting CUDA OOM switch to using enforce_eager = True

WARNING 06-04 18:34:00 config.py:1086] Casting torch.bfloat16 to torch.float16.


INFO 06-04 18:34:00 config.py:379] Using fp8 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. But it may cause slight accuracy drop without scaling factors. FP8_E5M2 (without scaling) is only supported on cuda version greater than 11.8. On ROCm (AMD GPU), FP8_E4M3 is instead supported for common inference criteria.


2024-06-04 18:34:08,989	INFO worker.py:1749 -- Started a local Ray instance.


INFO 06-04 18:34:11 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='/scratch/aakash_ks.iitr/models/deepseek/', speculative_config=None, tokenizer='/scratch/aakash_ks.iitr/models/deepseek/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=fp8, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/scratch/aakash_ks.iitr/models/deepseek/)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

INFO 06-04 18:34:31 distributed_gpu_executor.py:45] # GPU blocks: 4305, # CPU blocks: 1092


INFO 06-04 18:34:33 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.


INFO 06-04 18:34:33 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.


(RayWorkerWrapper pid=39626) INFO 06-04 18:34:33 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(RayWorkerWrapper pid=39626) INFO 06-04 18:34:33 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-04 18:34:33 custom_all_reduce.py:246] Registering 61 cuda graph addresses


INFO 06-04 18:34:33 model_runner.py:1017] Graph capturing finished in 0 secs.


In [9]:
import gc
import torch
# torch.backends.cuda.enable_mem_efficient_sdp(False)

In [10]:
import pandas as pd
from tqdm import tqdm
import math
import random

In [11]:
def naive_parse(answer):
    out = []
    start = False
    end = False
    for l in reversed(list(answer)):
        if l in '0123456789' and not end:
            start = True
            out.append(l)
        else:
            if start:
                end = True
        
    out = reversed(out)
    return ''.join(out)

In [12]:
import re
import sys
import subprocess

def return_last_print(output, n):
    lines = output.strip().split('\n')
    if lines:
        return lines[n]
    else:
        return ""

def process_code(code, return_shell_output=False):
    
    def repl(match):
        if "real" not in match.group():
            return "{}{}".format(match.group()[:-1], ', real=True)')
        else:
            return "{}{}".format(match.group()[:-1], ')')
    code = re.sub(r"symbols\([^)]+\)", repl, code)

    if return_shell_output:
        code = code.replace('\n', '\n    ')
            # Add a try...except block
        code = "\ntry:\n    from sympy import *\n{}\nexcept Exception as e:\n    print(e)\n    print('FAIL')\n".format(code)
    
    if not return_shell_output:
        print(code)
    with open('code.py', 'w') as fout:
        fout.write(code)
    
    batcmd = 'timeout 7 ' + sys.executable + ' code.py'
    try:
        shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
        return_value = return_last_print(shell_output, -1)
        print(shell_output)
        if return_shell_output:
            if return_value=='FAIL':
                CODE_STATUS = False
                return_value = return_last_print(shell_output, -2)
                if "not defined" in return_value:
                    return_value+='\nTry checking the formatting and imports'
            else:
                CODE_STATUS = True
            return return_value, CODE_STATUS  
        code_output = round(float(eval(return_value))) % 1000
    except Exception as e:
        print(e,'shell_output')
        code_output = -1
    
    if return_shell_output:
        if code_output==-1:
            CODE_STATUS = False
        else:
            CODE_STATUS = True
        return code_output, CODE_STATUS  
    
    
    return code_output


def process_text_output(output):
    result = output    
    try:
        result_output = re.findall(r'\\boxed\{(\d+)\}', result)

        print('BOXED', result_output)
        if not len(result_output):
            result_output = naive_parse(result)
        else:
            result_output = result_output[-1]

        print('BOXED FINAL', result_output)
        if not len(result_output):
            result_output = -1
        
        else:
            result_output = round(float(eval(result_output))) % 1000
    
    except Exception as e:
        print(e)
        print('ERROR PARSING TEXT')
        result_output = -1
    
    return result_output


In [13]:
#torch.cuda.empty_cache()
gc.collect()

32

In [14]:
code = """Below is a math problem you are to solve (positive numerical answer):
\"{}\"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""


cot = """Below is a math problem you are to solve (positive numerical answer!):
\"{}\"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

promplt_options = [code,cot]

In [15]:
tokenizer = llm.get_tokenizer()

In [16]:
import re
from collections import defaultdict
from collections import Counter

from numpy.random import choice
import numpy as np

tool_instruction = '\n\nPlease integrate natural language reasoning with programs to solve the above problem, and put your final numerical answer within \\boxed{}.\nNote that the intermediary calculations may be real numbers, but the final numerical answer would always be an integer.'


#tool_instruction = " The answer should be given as a non-negative modulo 1000."
#tool_instruction += '\nPlease integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}.'

temperature = 0.9
top_p = 1.0

temperature_coding = 0.9
top_p_coding = 1.0

   
total_results = {}
total_answers = {}
best_stats = {}
total_outputs = {}
question_type_counts = {}
starting_counts = (2,3)


for i, (test, sample_submission) in tqdm(enumerate(iter_test)):
    print(f"Solving problem {i} ...")
    TIME_SPENT = time.time() - NOTEBOOK_START_TIME

    if TIME_SPENT>TIME_LIMIT:
        break
    
    Q_START_TIME = time.time()
        
    for jj in tqdm(range(n_repetitions)):   
        problem = test['problem'].values[0]
        print(f"\n\n\nQUESTION {i} - {jj} - TIME_SPENT : {TIME_SPENT:.0f} secs")
        
        best, best_count = best_stats.get(i,(-1,-1))
        if best_count>np.sqrt(jj):
            print("SKIPPING CAUSE ALREADY FOUND BEST")
            continue
        
        if (time.time() - Q_START_TIME) > PER_Q_TIME_LIMIT:
            break
            
        outputs = total_outputs.get(i,[])
        text_answers, code_answers = question_type_counts.get(i,starting_counts)
        results = total_results.get(i,[])
        answers = total_answers.get(i,[])
        
        gc.collect()        ##

        try:
            ALREADY_GEN = 0
            code_error = None
            code_error_count = 0
            code_output = -1
            #initail_message = problem  + tool_instruction 
            counts = np.array([text_answers,code_answers])

            draw = choice(promplt_options, 1,
                          p=counts/counts.sum())

            initail_message = draw[0].format(problem,"{}")            
            prompt = f"User: {initail_message}"

            current_printed = len(prompt)
            print(f"{jj}_{prompt}\n")

            ######################
            model_inputs = tokenizer(prompt, return_tensors='pt')
            input_len = len(model_inputs['input_ids'][0])
            input_len2 = len(prompt)

            generation_output = llm.generate([prompt], SamplingParams(
                stop=stop_words, temperature=temperature, max_tokens=TOTAL_TOKENS-ALREADY_GEN, top_p=top_p, 
                include_stop_str_in_output=True))
            
            # output_ids = generation_output[0]
            
            decoded_output = prompt + generation_output[0].outputs[0].text
            
            ######################
            
            print(f"{decoded_output[current_printed:]}\n")
            current_printed += len(decoded_output[current_printed:])
            cummulative_code = ""
            
            
            stop_word_cond = False
            for stop_word in stop_words:
                stop_word_cond = stop_word_cond or (decoded_output[-len(stop_word):]==stop_word)
                
            
            while (stop_word_cond) and (ALREADY_GEN<(TOTAL_TOKENS)):

                if (decoded_output[-len("```python"):]=="```python"):
                    temperature_inner=temperature_coding
                    top_p_inner = top_p_coding
                    prompt = decoded_output
                else:
                    temperature_inner=temperature
                    top_p_inner = top_p
                    try:
                        if (decoded_output[-len("``````output"):]=="``````output"):
                            code_text = decoded_output.split('```python')[-1].split("``````")[0]
                        else:
                            code_text = decoded_output.split('```python')[-1].split("```")[0]
                        

                        cummulative_code+=code_text
                        code_output, CODE_STATUS = process_code(cummulative_code, return_shell_output=True)
                        print('CODE RESULTS', code_output)

                        if code_error==code_output:
                            code_error_count+=1
                        else:
                            code_error=code_output
                            code_error_count = 0

                        if not CODE_STATUS:
                            cummulative_code = cummulative_code[:-len(code_text)]

                            if code_error_count>=1:
                                print("REPEATED ERRORS")
                                break

                    except Exception as e:
                        print(e)
                        print('ERROR PARSING CODE')
                        code_output = -1

                    if code_output!=-1:
                        if (decoded_output[-len(")\n```"):]==")\n```"):
                            prompt = decoded_output+'```output\n'+str(code_output)+'\n```\n'
                        else:
                            prompt = decoded_output+'\n'+str(code_output)+'\n```\n'
                    else:
                        prompt = decoded_output
                        cummulative_code=""

                ######################

                model_inputs = tokenizer(prompt, return_tensors='pt')
                ALREADY_GEN =  len(model_inputs['input_ids'][0])-input_len
                
                generation_output = llm.generate([prompt], SamplingParams(
                    stop=stop_words, temperature=temperature_inner, max_tokens=TOTAL_TOKENS-ALREADY_GEN, top_p=top_p_inner, 
                    include_stop_str_in_output=True))
                
                decoded_output = prompt + generation_output[0].outputs[0].text
                
                ######################
                
                print(f"\nINTERMEDIATE OUT :\n{decoded_output[current_printed:]}\n")
                current_printed+=len(decoded_output[current_printed:])
                
                stop_word_cond = False
                for stop_word in stop_words:
                    stop_word_cond = stop_word_cond or (decoded_output[-len(stop_word):]==stop_word)
         
            raw_output = decoded_output[input_len2:]
            
            #print(f"\n\nOutput :\n{raw_output}\n")                            
            result_output = process_text_output(raw_output)
            
            try:
                code_output = round(float(eval(code_output))) % 1000
            except Exception as e:
                print(e,'final_eval')
                code_output = -1

        except Exception as e:
            print(e,"5")
            result_output, code_output = -1, -1

        if code_output!=-1:
            outputs.append(code_output)
            code_answers+=1

        if result_output!=-1:
            outputs.append(result_output)
            text_answers+=1

        if len(outputs) > 0:
            occurances = Counter(outputs).most_common()
            print(occurances)
            if occurances[0][1] > best_count:
                print("GOOD ANSWER UPDATED!")
                best = occurances[0][0]
                best_count = occurances[0][1]
            if occurances[0][1] > 5:
                print("ANSWER FOUND!")
                break

        results.append(result_output)
        answers.append(code_output)
        
        best_stats[i] = (best, best_count) 
        question_type_counts[i] = (text_answers, code_answers)
        total_outputs[i] = outputs
        
        total_results[i] = results
        total_answers[i] = answers

        print("code_answers",code_answers-starting_counts[1],"text_answers",text_answers-starting_counts[0])
        if DEBUG:
            break
            
    print(f"Predicted best answer: {best_stats}")
    sample_submission['answer'] = best_stats[i][0]
    env.predict(sample_submission)

0it [00:00, ?it/s]

Solving problem 0 ...


  0%|                                                                              | 0/10 [00:00<?, ?it/s]




QUESTION 0 - 0 - TIME_SPENT : 36 secs
0_User: Below is a math problem you are to solve (positive numerical answer!):
"Each of the three-digits numbers $111$ to $999$ is coloured blue or yellow in such a way that the sum of any two (not necessarily different) yellow numbers is equal to a blue number. What is the maximum possible number of yellow numbers there can be?"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \boxed{}.





Processed prompts:   0%|                                                            | 0/1 [00:00<?, ?it/s]

...

100%|█████████████████████████████████████████████████████████████████████| 10/10 [01:07<00:00,  3.85s/it]

100%|█████████████████████████████████████████████████████████████████████| 10/10 [01:07<00:00,  6.76s/it]


10it [14:42, 74.44s/it]

10it [14:42, 88.21s/it]

There are $6^4 = 1296$ possible outcomes when we roll four 6-sided dice.
To find the probability that the highest roll is a 5, we need to find the number of outcomes where the highest roll is a 5.
This is equivalent to finding the number of outcomes where all four dice show a number less than or equal to 5.
Each die can show a number from 1 to 5, so there are $5^4 = 625$ such outcomes.
Therefore, the probability that the highest roll is a 5 is $\frac{625}{1296}$.
To find $a + b$, we simply add the numerator and the denominator of the probability.
So, $a + b = 625 + 1296 = 1921$.
The answer is: $1921$.

BOXED []
BOXED FINAL 1921
eval() arg 1 must be a string, bytes or code object final_eval
[(449, 3), (97, 2), (0, 2), (297, 1), (55, 1), (921, 1)]
code_answers 4 text_answers 6
Predicted best answer: {0: (11, 2), 1: (391, 2), 2: (256, 3), 3: (1, 3), 4: (200, 2), 5: (36, 2), 6: (1, 4), 7: (310, 4), 8: (100, 3), 9: (449, 3)}


In [17]:
if not PRIVATE:
    display(env.df)
    print(f"\ncorrect: {(env.df['ground_truth'] == env.df['answer']).sum()} out of {len(env.df)}")

,index,id,problem,answer,ground_truth
0,1,246d26,Each of the three-digits numbers $111$ to $999...,11,250
1,9,d7e9c9,A function $f: \mathbb N \to \mathbb N$ satisf...,391,199
2,7,8ee6f3,"The points $\left(x, y\right)$ satisfying $((\...",256,320
3,2,2fc4ad,Let the `sparkle' operation on positive intege...,1,702
4,3,430b63,What is the minimum value of $5x^2+5y^2-8xy$ w...,200,800
5,0,229ee8,"Let $k, l > 0$ be parameters. The parabola $y ...",36,52
6,8,bedda4,Let $ABCD$ be a unit square. Let $P$ be the po...,1,480
7,4,5277ed,There exists a unique increasing geometric seq...,310,211
8,5,739bc9,For how many positive integers $m$ does the eq...,100,199
9,6,82e2a0,Suppose that we roll four 6-sided fair dice wi...,449,185



correct: 0 out of 10


In [18]:
with open('code.py', 'w') as fout:
    fout.write("print('done')")

batcmd = 'timeout 7 ' + sys.executable + ' code.py'
try:
    shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
    print(shell_output)
except:
    pass

done



In [19]:
time.time() - NOTEBOOK_START_TIME

919.1974251270294